In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 패키지 로드

In [11]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 데이터 백업

In [10]:
sw_data = pd.read_csv('c:/Users/Admin/AI/downloads/카드소비데이터 수원시/SUWON_S_DATA_TABLE_GENDER_SUM.csv')
sw_data = sw_data[['TA_YMD', 'HOUR', 'DAY', 'DONG', 'CNT', 'UNIT', 'TEMP', 'RAIN', 'AMT']]
sw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101513 entries, 0 to 101512
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   TA_YMD  101513 non-null  int64  
 1   HOUR    101513 non-null  int64  
 2   DAY     101513 non-null  int64  
 3   DONG    101513 non-null  object 
 4   CNT     101513 non-null  int64  
 5   UNIT    101513 non-null  int64  
 6   TEMP    101513 non-null  float64
 7   RAIN    101513 non-null  float64
 8   AMT     101513 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 7.0+ MB


# 모델 학습

In [8]:
sw_X = sw_data.iloc[:,:-1]
sw_y = sw_data.iloc[:,-1]
sw_X.shape, sw_y.shape

((101513, 8), (101513,))

In [12]:
le = LabelEncoder()

sw_X['DONG_code'] = le.fit_transform(sw_X['DONG'])   # 새 숫자 컬럼
sw_X = sw_X.drop(columns=['DONG'])  

In [13]:
tscv = TimeSeriesSplit(n_splits=5)

for train, test in tscv.split(sw_X):
    X_train, X_test = sw_X.iloc[train], sw_X.iloc[test]
    y_train, y_test = sw_y.iloc[train], sw_y.iloc[test]

    model = XGBRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print("MAE:", mean_absolute_error(y_test, pred))

MAE: 61969.80921891931
MAE: 52584.15421381469
MAE: 41544.993122032734
MAE: 43963.04344594928
MAE: 42599.578318370295


In [35]:
import pandas as pd
import numpy as np

# ---------------------------
# 더미 가게 매출 데이터 생성
# ---------------------------
np.random.seed(42)

# 최근 60일 (사장님이 입력했다고 가정)
dates = pd.date_range("2025-10-01", "2025-12-20", freq="D")

# 가게 일매출 (임의 생성, 현실적인 범위)
# 평일 낮고 / 주말 조금 높게
sales = []
for d in dates:
    base = 120_000
    if d.weekday() >= 5:  # 주말
        base *= 1.2
    noise = np.random.normal(0, 20_000)
    sales.append(max(20_000, base + noise))

store_df = pd.DataFrame({
    "ta_ymd": dates.strftime("%Y%m%d").astype(int),
    "store_amt": sales
})

# 저장 경로 (너가 쓰던 경로 그대로)
out_path = r"c:/Users/Admin/AI/downloads/카드소비데이터 수원시/store_sales.csv"
store_df.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"store_sales.csv 생성 완료 -> {out_path}")
print(store_df.head())


store_sales.csv 생성 완료 -> c:/Users/Admin/AI/downloads/카드소비데이터 수원시/store_sales.csv
     ta_ymd      store_amt
0  20251001  129934.283060
1  20251002  117234.713977
2  20251003  132953.770762
3  20251004  174460.597128
4  20251005  139316.932506


In [47]:
# -*- coding: utf-8 -*-
"""
Option 2 (추천)
- 모델 피처: temp + rain_mean + rain_max + 파생(temp_clip, log) + calendar + holiday + lag/rolling
- 고객 출력: rain_max(피크) 중심으로 안내
- A 검증: 데이터 범위 내(<= daily_max_ymd)면 데이터에서 날씨 사용
- B 운영 준비: 데이터 범위 밖이면 getVilageFcst로 temp/rain_mean/rain_max 생성(준비만)
"""

import os
import re
import numpy as np
import pandas as pd
import requests
import joblib
from dataclasses import dataclass
from sklearn.metrics import mean_absolute_error

# =========================
# Config
# =========================
@dataclass
class Config:
    dong_data_path: str = r"c:/Users/Admin/AI/downloads/카드소비데이터 수원시/SUWON_S_DATA_TABLE_GENDER_SUM.csv"
    store_sales_path: str = r"c:/Users/Admin/AI/downloads/카드소비데이터 수원시/store_sales.csv"

    model_out_dir: str = r"c:/Users/Admin/AI/downloads/카드소비데이터 수원시/models"
    model_name: str = "suwon_daily_amt_meanmax.pkl"

    train_end: int = 20241231
    test_start: int = 20250101

    lags: tuple = (1, 7, 14, 28)
    rolling_windows: tuple = (7, 14, 28)

    use_log_target: bool = True

    share_window_days: int = 56
    min_overlap_days: int = 7

    # simple adjustment
    adj_weekend: float = 1.05
    adj_holiday: float = 1.08
    adj_rain: float = 0.93

    # KMA key env name
    service_key_env: str = "RAIN_ID"

CFG = Config()

# (예보용) nx, ny 매핑 (일단 동일)
DONG_TO_GRID = {k: (61, 121) for k in ["고등동","인계동","행궁동","매산동","매교동","우만1동","우만2동","지동","화서1동","화서2동"]}


# =========================
# Metrics
# =========================
def wmape(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.maximum(np.sum(np.abs(y_true)), eps)
    return float(np.sum(np.abs(y_true - y_pred)) / denom * 100.0)


# =========================
# Holidays (2023-01-01 ~ 2025-10-31)
# =========================
def holiday_set_2023_2025_upto_oct31() -> set[pd.Timestamp]:
    dates = []
    dates += ["2023-01-01","2023-01-21","2023-01-22","2023-01-23","2023-01-24","2023-03-01",
              "2023-05-05","2023-05-27","2023-05-29","2023-06-06","2023-08-15","2023-09-28",
              "2023-09-29","2023-09-30","2023-10-03","2023-10-09","2023-12-25"]
    dates += ["2024-01-01","2024-02-09","2024-02-10","2024-02-11","2024-02-12","2024-03-01",
              "2024-05-05","2024-05-06","2024-05-15","2024-06-06","2024-08-15","2024-09-16",
              "2024-09-17","2024-09-18","2024-10-03","2024-10-09","2024-12-25"]
    dates += ["2025-01-01","2025-01-28","2025-01-29","2025-01-30","2025-03-01","2025-03-03",
              "2025-05-05","2025-05-06","2025-06-06","2025-08-15",
              "2025-10-03","2025-10-05","2025-10-06","2025-10-07","2025-10-08","2025-10-09"]
    s = set(pd.to_datetime(dates))
    start, end = pd.Timestamp("2023-01-01"), pd.Timestamp("2025-10-31")
    return {d for d in s if start <= d <= end}

HOLIDAYS = holiday_set_2023_2025_upto_oct31()


# =========================
# Helpers
# =========================
def ymd_int_to_ts(ymd: int) -> pd.Timestamp:
    return pd.to_datetime(str(int(ymd)), format="%Y%m%d")

def make_calendar_features(date: pd.Timestamp) -> dict:
    # ✅ 0~6 체계로 고정 (중복/혼선 방지)
    dow = int(date.dayofweek)  # 0=Mon ... 6=Sun
    is_weekend = int(dow in [5, 6])
    m = date.month
    return {
        "dow": dow,
        "is_weekend": is_weekend,
        "year": date.year,
        "month": m,
        "day": date.day,
        "weekofyear": int(date.isocalendar().week),
        "month_sin": float(np.sin(2*np.pi*m/12.0)),
        "month_cos": float(np.cos(2*np.pi*m/12.0)),
        "dow_sin": float(np.sin(2*np.pi*dow/7.0)),
        "dow_cos": float(np.cos(2*np.pi*dow/7.0)),
    }

def make_holiday_features(date: pd.Timestamp) -> dict:
    is_h = int(date in HOLIDAYS)
    is_before = int((date + pd.Timedelta(days=1)) in HOLIDAYS)
    is_after  = int((date - pd.Timedelta(days=1)) in HOLIDAYS)
    around = int(any((date + pd.Timedelta(days=k)) in HOLIDAYS for k in [-2,-1,0,1,2]))
    return {
        "is_holiday": is_h,
        "is_before_holiday": is_before,
        "is_after_holiday": is_after,
        "is_around_holiday_2d": around
    }

def make_weather_derived_features(temp: float, rain_mean: float, rain_max: float) -> dict:
    """✅ 원본 temp/rain_mean/rain_max는 daily에 이미 있으므로 여기서는 '파생'만 만든다."""
    temp = float(temp)
    rain_mean = max(0.0, float(rain_mean))
    rain_max = max(0.0, float(rain_max))
    return {
        "temp_clip": float(np.clip(temp, -20, 45)),
        "is_rain": int(max(rain_mean, rain_max) > 0),
        "rain_mean_log1p": float(np.log1p(rain_mean)),
        "rain_max_log1p": float(np.log1p(rain_max)),
    }

def adjust_factor(is_weekend: bool, is_holiday: bool, is_rain: bool, cfg: Config) -> float:
    adj = 1.0
    if is_weekend: adj *= cfg.adj_weekend
    if is_holiday: adj *= cfg.adj_holiday
    if is_rain:    adj *= cfg.adj_rain
    return float(adj)


# =========================
# 1) Load raw + daily aggregation
# =========================
def load_raw(cfg: Config) -> pd.DataFrame:
    df = pd.read_csv(cfg.dong_data_path)
    rename_map = {
        "TA_YMD": "ta_ymd",
        "HOUR": "hour",
        "DAY": "dow_raw",
        "DONG": "dong",
        "AMT": "amt",
        "CNT": "cnt",
        "UNIT": "unit",
        "TEMP": "temp",
        "RAIN": "rain",
    }
    df = df.rename(columns=rename_map)

    needed = {"ta_ymd","hour","dong","amt","cnt","unit","temp","rain"}
    miss = needed - set(df.columns)
    if miss:
        raise ValueError(f"Missing columns: {miss}")

    df["ta_ymd"] = pd.to_numeric(df["ta_ymd"], errors="raise").astype(int)
    df["date"] = pd.to_datetime(df["ta_ymd"].astype(str), format="%Y%m%d", errors="raise")
    df["hour"] = pd.to_numeric(df["hour"], errors="coerce").astype(int)
    df["dong"] = df["dong"].astype(str)

    for c in ["amt","cnt","unit","temp","rain"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df.sort_values(["dong","date","hour"]).reset_index(drop=True)

def make_daily(df: pd.DataFrame) -> pd.DataFrame:
    """
    - amt: 일합
    - cnt: 일합
    - temp: 일평균
    - rain: 시간대 평균 강수량(네 정의) -> 일평균(rain_mean), 일최대(rain_max)
    """
    daily = df.groupby(["dong","ta_ymd","date"], as_index=False).agg(
        amt=("amt","sum"),
        cnt=("cnt","sum"),
        temp=("temp","mean"),
        rain_mean=("rain","mean"),
        rain_max=("rain","max"),
    )
    daily["unit"] = (daily["amt"] / daily["cnt"].replace(0, np.nan)).fillna(0)
    return daily.sort_values(["dong","ta_ymd"]).reset_index(drop=True)


# =========================
# 2) lag/rolling
# =========================
def add_lag_rolling(daily: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    g = daily.groupby("dong", sort=False)

    for lag in cfg.lags:
        daily[f"amt_lag_{lag}"] = g["amt"].shift(lag)
        daily[f"cnt_lag_{lag}"] = g["cnt"].shift(lag)
        daily[f"unit_lag_{lag}"] = g["unit"].shift(lag)

    for w in cfg.rolling_windows:
        minp = max(3, w//3)
        daily[f"amt_roll_{w}_mean"] = g["amt"].shift(1).rolling(w, min_periods=minp).mean()
        daily[f"amt_roll_{w}_std"]  = g["amt"].shift(1).rolling(w, min_periods=minp).std()

    return daily


# =========================
# 3) dataset build (✅ 중복 컬럼 완전 방지)
# =========================
def build_dataset(daily: pd.DataFrame, cfg: Config):
    cal_df = pd.DataFrame(list(daily["date"].apply(make_calendar_features)))
    hol_df = pd.DataFrame(list(daily["date"].apply(make_holiday_features)))
    wea_df = pd.DataFrame(list(daily.apply(lambda r: make_weather_derived_features(r["temp"], r["rain_mean"], r["rain_max"]), axis=1)))

    out = pd.concat([daily.reset_index(drop=True), cal_df, hol_df, wea_df], axis=1)

    out = add_lag_rolling(out, cfg)

    # ✅ 중복 컬럼 검사 (문제 있으면 여기서 바로 알림)
    dup = out.columns[out.columns.duplicated()].tolist()
    if dup:
        raise ValueError(f"Duplicate columns detected: {dup}")

    feature_cols = [
        "dong",
        # calendar
        "dow","is_weekend","year","month","day","weekofyear",
        "month_sin","month_cos","dow_sin","dow_cos",
        # raw weather
        "temp",
        "rain_mean","rain_max",
        # derived weather
        "temp_clip","is_rain","rain_mean_log1p","rain_max_log1p",
        # holiday
        "is_holiday","is_before_holiday","is_after_holiday","is_around_holiday_2d",
    ]
    for lag in cfg.lags:
        feature_cols += [f"amt_lag_{lag}", f"cnt_lag_{lag}", f"unit_lag_{lag}"]
    for w in cfg.rolling_windows:
        feature_cols += [f"amt_roll_{w}_mean", f"amt_roll_{w}_std"]

    data = out.dropna(subset=feature_cols + ["amt"]).copy()

    if cfg.use_log_target:
        data["target"] = np.log1p(data["amt"])
    else:
        data["target"] = data["amt"]

    return data, feature_cols


# =========================
# 4) train (LightGBM)
# =========================
def train_and_eval(data: pd.DataFrame, feature_cols, cfg: Config):
    train = data[data["ta_ymd"] <= cfg.train_end].copy()
    test  = data[data["ta_ymd"] >= cfg.test_start].copy()

    if train.empty or test.empty:
        raise ValueError("train/test split empty. 날짜 범위 확인")

    # time-aware validation split
    tr_sorted = train.sort_values("ta_ymd")
    split = int(len(tr_sorted) * 0.8)
    tr_idx = tr_sorted.index[:split]
    va_idx = tr_sorted.index[split:]

    X_train = train.loc[tr_idx, feature_cols].copy()
    y_train = train.loc[tr_idx, "target"].values
    X_valid = train.loc[va_idx, feature_cols].copy()
    y_valid = train.loc[va_idx, "target"].values

    X_test = test[feature_cols].copy()
    y_test = test["target"].values

    # categorical
    X_train["dong"] = X_train["dong"].astype("category")
    X_valid["dong"] = X_valid["dong"].astype("category")
    X_test["dong"] = X_test["dong"].astype("category")

    import lightgbm as lgb

    model = lgb.LGBMRegressor(
        n_estimators=5000,
        learning_rate=0.02,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="l1",
        callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)]
    )

    pred = model.predict(X_test)

    if cfg.use_log_target:
        y_true = np.expm1(y_test)
        y_pred = np.expm1(pred)
    else:
        y_true = y_test
        y_pred = pred

    mae = mean_absolute_error(y_true, y_pred)
    w = wmape(y_true, y_pred)

    print("[Model] LightGBM (rain_mean + rain_max)")
    print(f"[Test] MAE   : {mae:,.0f}")
    print(f"[Test] WMAPE : {w:.2f}%")

    os.makedirs(cfg.model_out_dir, exist_ok=True)
    out_path = os.path.join(cfg.model_out_dir, cfg.model_name)
    joblib.dump({"model": model, "feature_cols": feature_cols, "cfg": cfg}, out_path)
    print(f"Saved -> {out_path}")

    return out_path


# =========================
# 5) (B 준비) forecast weather (VilageFcst)
# =========================
def _get_service_key(cfg: Config) -> str:
    k = os.getenv(cfg.service_key_env)
    if not k:
        raise ValueError(f"환경변수 {cfg.service_key_env}가 없습니다. .env에 {cfg.service_key_env}=... 넣어주세요.")
    return k

def _pick_base_datetime_kst():
    now = pd.Timestamp.now(tz="Asia/Seoul")
    base_times = ["2300","2000","1700","1400","1100","0800","0500","0200"]
    for bt in base_times:
        cand = pd.Timestamp(f"{now.strftime('%Y-%m-%d')} {bt[:2]}:{bt[2:]}", tz="Asia/Seoul")
        if now >= cand + pd.Timedelta(minutes=40):
            return now.strftime("%Y%m%d"), bt
    yday = (now - pd.Timedelta(days=1)).strftime("%Y%m%d")
    return yday, "2300"

def _parse_pcp(v: str) -> float:
    if v is None: return 0.0
    v = str(v).strip()
    if v == "" or "강수없음" in v: return 0.0
    if "미만" in v: return 0.5
    if "~" in v: v = v.split("~")[0]
    m = re.findall(r"[-+]?\d*\.?\d+", v)
    return float(m[0]) if m else 0.0

def get_forecast_weather(dong: str, target_ymd: int, cfg: Config):
    sk = _get_service_key(cfg)
    base_date, base_time = _pick_base_datetime_kst()

    if dong not in DONG_TO_GRID:
        raise ValueError(f"동 nx/ny 매핑 없음: {dong}")

    nx, ny = DONG_TO_GRID[dong]
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    full_url = f"{url}?serviceKey={sk}"

    params = {
        "pageNo": 1,
        "numOfRows": 2000,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": int(nx),
        "ny": int(ny),
    }

    r = requests.get(full_url, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()

    header = data["response"]["header"]
    if header.get("resultCode") != "00":
        raise ValueError(f"KMA Vilage error: {header.get('resultMsg')} (code={header.get('resultCode')})")

    items = data["response"]["body"]["items"]["item"]

    temps, pcps = [], []
    for it in items:
        if it.get("fcstDate") != str(target_ymd):
            continue
        if it.get("category") == "TMP":
            temps.append(float(it.get("fcstValue")))
        elif it.get("category") == "PCP":
            pcps.append(_parse_pcp(it.get("fcstValue")))

    if not temps:
        raise ValueError(f"예보(TMP) 없음: target={target_ymd} base={base_date}{base_time}")

    temp = float(np.mean(temps))
    rain_mean = float(np.mean(pcps)) if pcps else 0.0
    rain_max = float(np.max(pcps)) if pcps else 0.0
    return temp, rain_mean, rain_max


# =========================
# 6) store share + inference + customer demo
# =========================
def load_store_sales(path: str) -> pd.DataFrame:
    s = pd.read_csv(path)
    if "ta_ymd" not in s.columns or "store_amt" not in s.columns:
        raise ValueError("store_sales.csv needs columns: ta_ymd, store_amt")
    s["ta_ymd"] = pd.to_numeric(s["ta_ymd"], errors="coerce").astype(int)
    s["date"] = pd.to_datetime(s["ta_ymd"].astype(str), format="%Y%m%d", errors="raise")
    s["store_amt"] = pd.to_numeric(s["store_amt"], errors="coerce")
    return s.dropna(subset=["store_amt"]).sort_values("ta_ymd").reset_index(drop=True)

def compute_share(daily: pd.DataFrame, store: pd.DataFrame, dong: str, target_ymd: int, cfg: Config) -> float:
    end_date = ymd_int_to_ts(target_ymd) - pd.Timedelta(days=1)
    start_date = end_date - pd.Timedelta(days=cfg.share_window_days)

    dong_hist = daily[(daily["dong"]==dong) & (daily["date"]>=start_date) & (daily["date"]<=end_date)][["date","amt"]]
    store_hist = store[(store["date"]>=start_date) & (store["date"]<=end_date)][["date","store_amt"]]
    merged = dong_hist.merge(store_hist, on="date", how="inner")

    if len(merged) < cfg.min_overlap_days:
        return 0.0

    denom = merged["amt"].sum()
    if denom <= 0:
        return 0.0

    share = float(merged["store_amt"].sum() / denom)
    return float(np.clip(share, 0.0, 0.2))

def build_one_row_for_predict(daily: pd.DataFrame, dong: str, target_ymd: int,
                              temp: float, rain_mean: float, rain_max: float, cfg: Config,
                              feature_cols: list) -> pd.DataFrame:
    # dong별 과거
    h = daily[daily["dong"]==dong].copy().sort_values("ta_ymd")

    target_date = ymd_int_to_ts(target_ymd)
    cal = make_calendar_features(target_date)
    hol = make_holiday_features(target_date)
    wea_der = make_weather_derived_features(temp, rain_mean, rain_max)

    # target row 추가
    row = {
        "dong": dong,
        "ta_ymd": int(target_ymd),
        "date": target_date,
        "amt": np.nan,
        "cnt": np.nan,
        "unit": np.nan,
        "temp": float(temp),
        "rain_mean": float(rain_mean),
        "rain_max": float(rain_max),
    }
    tmp = pd.concat([h, pd.DataFrame([row])], ignore_index=True).sort_values("ta_ymd").reset_index(drop=True)

    # lag/rolling
    g = tmp.groupby("dong", sort=False)
    for lag in cfg.lags:
        tmp[f"amt_lag_{lag}"] = g["amt"].shift(lag)
        tmp[f"cnt_lag_{lag}"] = g["cnt"].shift(lag)
        tmp[f"unit_lag_{lag}"] = g["unit"].shift(lag)

    for w in cfg.rolling_windows:
        minp = max(3, w//3)
        tmp[f"amt_roll_{w}_mean"] = g["amt"].shift(1).rolling(w, min_periods=minp).mean()
        tmp[f"amt_roll_{w}_std"]  = g["amt"].shift(1).rolling(w, min_periods=minp).std()

    last = tmp.iloc[[-1]].copy()
    # attach derived features
    for k,v in cal.items(): last[k] = v
    for k,v in hol.items(): last[k] = v
    for k,v in wea_der.items(): last[k] = v

    # final duplicate check
    dup = last.columns[last.columns.duplicated()].tolist()
    if dup:
        raise ValueError(f"Duplicate columns in inference row: {dup}")

    return last[feature_cols].copy()

def predict_customer_sales(dong: str, target_ymd: int, model_pack_path: str, cfg: Config):
    pack = joblib.load(model_pack_path)
    model = pack["model"]
    feature_cols = pack["feature_cols"]

    raw = load_raw(cfg)
    daily = make_daily(raw)
    daily_max_ymd = int(daily["ta_ymd"].max())

    # A 검증: 데이터 날씨 사용 / B 운영 준비: 예보 사용
    if target_ymd <= daily_max_ymd:
        row = daily[(daily["dong"]==dong) & (daily["ta_ymd"]==int(target_ymd))]
        if row.empty:
            raise ValueError(f"daily에 해당 dong/date 없음: {dong}, {target_ymd}")
        temp = float(row["temp"].iloc[0])
        rain_mean = float(row["rain_mean"].iloc[0])
        rain_max = float(row["rain_max"].iloc[0])
        weather_src = "DATA_WEATHER(A 검증)"
    else:
        temp, rain_mean, rain_max = get_forecast_weather(dong, target_ymd, cfg)
        weather_src = "KMA_FORECAST(B 운영)"

    # 동 예측
    X = build_one_row_for_predict(daily, dong, target_ymd, temp, rain_mean, rain_max, cfg, feature_cols)
    X["dong"] = X["dong"].astype("category")

    pred = model.predict(X)[0]
    dong_pred = float(np.expm1(pred) if cfg.use_log_target else pred)
    dong_pred = max(0.0, dong_pred)

    # store share + adjustment (가능하면)
    store_pred = None
    share = None
    adj = None
    try:
        store = load_store_sales(cfg.store_sales_path)
        share = compute_share(daily, store, dong, target_ymd, cfg)

        date_ts = ymd_int_to_ts(target_ymd)
        cal = make_calendar_features(date_ts)
        hol = make_holiday_features(date_ts)
        adj = adjust_factor(bool(cal["is_weekend"]), bool(hol["is_holiday"]), bool(rain_max>0 or rain_mean>0), cfg)
        store_pred = max(0.0, dong_pred * share * adj)
    except Exception:
        pass

    # 고객 메시지 (rain_max 중심)
    lines = []
    lines.append(f"📍지역(동): {dong}")
    lines.append(f"📅날짜: {target_ymd}")
    lines.append(f"🌦️날씨(출처): {weather_src}")
    lines.append(f"   - 평균기온(일평균): {temp:.1f}°C")
    lines.append(f"   - 시간대 평균 강수량(일평균): {rain_mean:.2f} mm")
    lines.append(f"   - ✅시간대 평균 강수량(일최대, 피크): {rain_max:.2f} mm  ← 고객에게 보여줄 핵심")
    lines.append(f"💰예상 동 전체 일매출: {dong_pred:,.0f} 원")

    if store_pred is None:
        lines.append("🏪가게 예측: store_sales.csv가 없거나 형식 오류 → 동 예측만 제공")
    else:
        lines.append(f"🏪가게 점유율(최근 {cfg.share_window_days}일): {share*100:.3f}%")
        lines.append(f"🏪보정계수(adj): {adj:.3f}")
        lines.append(f"🏪예상 가게 일매출: {store_pred:,.0f} 원")

    return {
        "dong": dong,
        "date": int(target_ymd),
        "temp": round(temp, 1),
        "rain_mean": round(rain_mean, 2),
        "rain_max": round(rain_max, 2),
        "dong_pred": int(dong_pred),
        "share_pct": None if share is None else round(share*100, 3),
        "store_pred": None if store_pred is None else int(store_pred),
        "customer_message": "\n".join(lines),
    }


# =========================
# MAIN: train + customer demo
# =========================
def main():
    raw = load_raw(CFG)
    daily = make_daily(raw)
    data, feature_cols = build_dataset(daily, CFG)

    model_path = train_and_eval(data, feature_cols, CFG)

    # ===== 고객 입력 시나리오 =====
    customer_dong = "고등동"
    customer_date = int(daily["ta_ymd"].max())  # 예: 20251031 (A 검증)

    result = predict_customer_sales(customer_dong, customer_date, model_path, CFG)

    print("\n" + "="*60)
    print("✅ [고객 화면 출력 예시]")
    print("="*60)
    print(result["customer_message"])
    print("="*60)
    print("\n[RAW RESULT]")
    print(result)

if __name__ == "__main__":
    main()


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000905 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5664
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 40
[LightGBM] [Info] Start training from score 16.047767
[Model] LightGBM (rain_mean + rain_max)
[Test] MAE   : 2,157,716
[Test] WMAPE : 12.73%
Saved -> c:/Users/Admin/AI/downloads/카드소비데이터 수원시/models\suwon_daily_amt_meanmax.pkl

✅ [고객 화면 출력 예시]
📍지역(동): 고등동
📅날짜: 20251031
🌦️날씨(출처): DATA_WEATHER(A 검증)
   - 평균기온(일평균): 13.6°C
   - 시간대 평균 강수량(일평균): 0.19 mm
   - ✅시간대 평균 강수량(일최대, 피크): 1.50 mm  ← 고객에게 보여줄 핵심
💰예상 동 전체 일매출: 3,082,178 원
🏪가게 점유율(최근 56일): 4.635%
🏪보정계수(adj): 0.930
🏪예상 가게 일매출: 132,851 원

[RAW RESULT]
{'dong': '고등동', 'date': 20251031, 'temp': 13.6, 'rain_mean': 0.19, 'rain_max': 1.5, 'dong_pred': 3082177, 'share_pct': 4.635, 'store_pred': 132850, 'customer_message': '📍지역(동): 고등동\n📅날짜: 20251031\n🌦️날씨(출처): D